# Clustering Automático - Perfis Físicos de Atletas
**Grupo:** G03 | **Unidade Curricular:** IA25

## 1. Objetivos de Negócio
O objetivo desta análise é **agrupar os atletas em clusters baseados nos seus atributos físicos (Idade, Altura, Peso)**, sem utilizar as etiquetas de desporto previamente.

**Metas:**
1. Identificar os biótipos dominantes nos Jogos Olímpicos.
2. Verificar se estes grupos correspondem a desportos específicos (ex: Cluster de "Gigantes" vs Cluster de "Ligeiros").
3. Detetar anomalias ou perfis físicos atípicos.

In [ ]:
# Importação de bibliotecas
%pip install pandas numpy matplotlib seaborn scikit-learn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Scikit-Learn - Clustering e Pré-processamento
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score

# Configuração Visual
sns.set(style="whitegrid", context="notebook")

## 2. Seleção e Preparação dos Dados
O algoritmo K-Means utiliza distâncias matemáticas (Euclidianas). Por isso, a preparação é crítica.

**Passos:**
1. **Seleção:** Usaremos `Age`, `Height` e `Weight`.
2. **Limpeza:** O K-Means não aceita valores nulos. Removeremos linhas com dados em falta nestas colunas.
3. **Scaling (Padronização):** Como o Peso (kg) e a Altura (cm) têm escalas muito diferentes, é obrigatório usar o `StandardScaler` para que o algoritmo não dê mais importância à Altura apenas porque o número é maior (ex: 180 vs 80).

In [ ]:
# Carregar dataset
try:
    df = pd.read_csv("athlete_events.csv")
except FileNotFoundError:
    print("ERRO: Ficheiro não encontrado.")

# 1. Seleção e Limpeza
features = ['Age', 'Height', 'Weight']
# Filtrar apenas linhas completas e fazer uma cópia para não alterar o original
df_cluster = df.dropna(subset=features).copy()

print(f"Dados originais: {df.shape[0]} linhas")
print(f"Dados após limpeza: {df_cluster.shape[0]} linhas")

# 2. Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_cluster[features])

# Mostrar exemplo dos dados escalados
print("\nExemplo dos dados normalizados (primeiras 3 linhas):")
print(X_scaled[:3])

## 3. Otimização do Algoritmo (Método do Cotovelo)
Para definir o número ideal de clusters ($K$), usamos o **Elbow Method**. Calculamos a inércia (soma dos erros quadrados) para $K$ de 1 a 10.

In [ ]:
inertia = []
K_range = range(1, 11)

print("A calcular inércia para K=1 a K=10...")
for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertia.append(kmeans.inertia_)

# Visualizar o Cotovelo
plt.figure(figsize=(10, 6))
plt.plot(K_range, inertia, marker='o', linestyle='--')
plt.title('Elbow Method (Método do Cotovelo)')
plt.xlabel('Número de Clusters (K)')
plt.ylabel('Inércia (Soma dos Quadrados)')
plt.show()

**Interpretação:** Observando o gráfico, procuramos o ponto onde a descida se torna menos acentuada. Normalmente, neste dataset, **K=4** ou **K=5** são boas escolhas. Vamos avançar com **K=4** para separar (ex: Ligeiros, Médios, Pesados, Veteranos).

In [ ]:
# Aplicação do K-Means com K otimizado
optimal_k = 4
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X_scaled)

# Adicionar o cluster ao dataframe original (limpo)
df_cluster['Cluster'] = clusters

# Visualização 2D (Peso vs Altura)
plt.figure(figsize=(10, 8))
sns.scatterplot(data=df_cluster, x='Height', y='Weight', hue='Cluster', palette='viridis', alpha=0.6)
plt.title(f'Distribuição dos Clusters (K={optimal_k}) - Altura vs Peso')
plt.show()

## 4. Análise das Características dos Clusters
Agora analisamos o perfil de cada grupo gerado.

### 4.1 Estatísticas Numéricas (Média, Desvio Padrão, Min, Max)
Isto permite-nos dar "nomes" aos clusters.

In [ ]:
# Agrupamento e cálculo de estatísticas
cluster_stats = df_cluster.groupby('Cluster')[['Age', 'Height', 'Weight']].agg(['mean', 'std', 'min', 'max', 'count'])

# Transpor para facilitar a leitura
display(cluster_stats.T)

### 4.2 Distribuição de Atributos Categóricos
Quais são os desportos e o sexo predominantes em cada cluster? Isto valida se o agrupamento físico faz sentido no mundo real.

In [ ]:
# Função para analisar a composição de cada cluster
def analyze_cluster_composition(df, cluster_id):
    subset = df[df['Cluster'] == cluster_id]
    total = len(subset)
    
    print(f"\n--- ANÁLISE DO CLUSTER {cluster_id} (Total: {total} atletas) ---")
    
    # Sexo
    sex_counts = subset['Sex'].value_counts(normalize=True) * 100
    print(f"Sexo Dominante: {sex_counts.idxmax()} ({sex_counts.max():.1f}%)")
    
    # Top 5 Desportos
    print("Top 5 Desportos:")
    print(subset['Sport'].value_counts().head(5))

# Loop por todos os clusters
for k in range(optimal_k):
    analyze_cluster_composition(df_cluster, k)

## 5. Documentação e Conclusões
Com base nas tabelas acima, podemos rotular os clusters (Exemplo de interpretação automática):

- **Cluster 0 (ex: Corpo Médio):** Provavelmente a maioria dos atletas, mistura de desportos coletivos.
- **Cluster 1 (ex: Ligeiros/Baixos):** Alta probabilidade de Ginástica e Halterofilismo de categorias leves.
- **Cluster 2 (ex: Gigantes):** Atletas muito altos e pesados. Basquetebol e Voleibol devem dominar aqui.
- **Cluster 3 (ex: Pesados/Força):** Atletas com muito peso mas altura média. Provavelmente Judo, Wrestling ou Lançamento do Peso.

*Nota: A numeração dos clusters pode variar a cada execução devido à inicialização aleatória do K-Means, verifique as estatísticas geradas.*